In [1]:
from spacebase import *
import sqlite3
import pandas as pd
import datetime

In [22]:
response = (make_request(myShips_url))

In [30]:
df = pd.DataFrame(response.json()['ships'])

In [31]:
df

,id,location,x,y,cargo,spaceAvailable,type,class,maxCargo,loadingSpeed,speed,manufacturer,plating,weapons
0,cl55thau617774315s68iz3fpjj,OE-KO,-49,1,"[{'good': 'FUEL', 'quantity': 31, 'totalVolume...",19,JW-MK-I,MK-I,50,25,1,Jackshaw,5,5
1,cl58tw4u525771615s6bnh9pips,OE-PM-TR,-20,5,"[{'good': 'METALS', 'quantity': 55, 'totalVolu...",25,GR-MK-I,MK-I,100,100,1,Gravager,10,5


In [ ]:
conn = sqlite3.connect('./spacetraders.db')
cursor = conn.cursor()

In [13]:
def sql_command(command: str):
    return cursor.execute(command)

# takes a list of data and returns a comma separated string
def sql_clean_values(values):
    for i in range(len(values)):
        if type(values[i]) == str:
            values[i] = f'"{values[i]}"'
    return ", ".join(values)

#  VALUES IS A LIST OF LISTS OF STRINGS
def sql_insert_into(table_name: str, keys: str, values: list):
    # if values is NOT a list of lists, wrap it in a list lol
    if type[values[0]] is not list: values = [values]

    sql_command(f"""INSERT INTO {table_name} ({keys}) 
        VALUES ({sql_clean_values(values)})""")
    return conn.commit()

# column_names and new_data are lists
def sql_update(table_name: str, column_names: list, new_data: list, condition: str):
    sql_command(f"""UPDATE {table_name}
        SET {", ".join([f"{key} = {value}" for key, value in zip(column_names, new_data)])}
        WHERE {condition}""")
    return conn.commit()

In [ ]:
"""
particulars(data subject to change)
user
flightPlan
marketplace
(my)ships
(my)structures

types (unchanging):
locations
goods
loans (types)
structures (types)
ships (types)
"""

In [52]:
import pandas
database = {"locations":pandas.DataFrame({}),"goods":pandas.DataFrame({}),"loans":pandas.DataFrame({}),"structures":pandas.DataFrame({}),"ships":pandas.DataFrame({}),
    "user":{},"flightPlans":pandas.DataFrame({}),"markets":pandas.DataFrame({}),"myShips":pandas.DataFrame({}),"myStructures":pandas.DataFrame({})}

In [48]:
response = 

In [50]:
response = pd.DataFrame(get_system_locations("OE"))
response

,symbol,type,name,x,y,allowsConstruction,traits,messages
0,OE-PM,PLANET,Prime,-19,3,False,"[METAL_ORES, SOME_NATURAL_CHEMICALS]",NaN
1,OE-PM-TR,MOON,Tritus,-20,5,False,[NATURAL_CHEMICALS],NaN
2,OE-CR,PLANET,Carth,4,-13,False,"[METAL_ORES, NATURAL_CHEMICALS, ABUNDANT_RARE_...",NaN
3,OE-KO,PLANET,Koria,-49,1,False,"[SOME_NATURAL_CHEMICALS, SOME_RARE_METAL_ORES]",NaN
4,OE-UC,PLANET,Ucarro,-17,-72,False,"[SOME_METAL_ORES, ARABLE_LAND]",NaN
5,OE-UC-AD,MOON,Ado,-15,-73,False,"[SOME_METAL_ORES, SOME_RARE_METAL_ORES, TECHNO...",NaN
6,OE-UC-OB,MOON,Obo,-17,-74,False,"[SOME_METAL_ORES, NATURAL_CHEMICALS, SOME_TECH...",NaN
7,OE-NY,ASTEROID,Nyon,43,-46,True,"[SOME_NATURAL_CHEMICALS, RARE_METAL_ORES]",NaN
8,OE-BO,GAS_GIANT,Bo,-59,60,True,"[SOME_HELIUM_3, TECHNOLOGICAL_RUINS]",NaN
9,OE-W-XV,WORMHOLE,Wormhole,5,-101,False,[],[Extensive research has revealed a partially f...


In [54]:
database['locations'].update(response)

In [55]:
database['locations']

""


In [56]:
response

,symbol,type,name,x,y,allowsConstruction,traits,messages
0,OE-PM,PLANET,Prime,-19,3,False,"[METAL_ORES, SOME_NATURAL_CHEMICALS]",NaN
1,OE-PM-TR,MOON,Tritus,-20,5,False,[NATURAL_CHEMICALS],NaN
2,OE-CR,PLANET,Carth,4,-13,False,"[METAL_ORES, NATURAL_CHEMICALS, ABUNDANT_RARE_...",NaN
3,OE-KO,PLANET,Koria,-49,1,False,"[SOME_NATURAL_CHEMICALS, SOME_RARE_METAL_ORES]",NaN
4,OE-UC,PLANET,Ucarro,-17,-72,False,"[SOME_METAL_ORES, ARABLE_LAND]",NaN
5,OE-UC-AD,MOON,Ado,-15,-73,False,"[SOME_METAL_ORES, SOME_RARE_METAL_ORES, TECHNO...",NaN
6,OE-UC-OB,MOON,Obo,-17,-74,False,"[SOME_METAL_ORES, NATURAL_CHEMICALS, SOME_TECH...",NaN
7,OE-NY,ASTEROID,Nyon,43,-46,True,"[SOME_NATURAL_CHEMICALS, RARE_METAL_ORES]",NaN
8,OE-BO,GAS_GIANT,Bo,-59,60,True,"[SOME_HELIUM_3, TECHNOLOGICAL_RUINS]",NaN
9,OE-W-XV,WORMHOLE,Wormhole,5,-101,False,[],[Extensive research has revealed a partially f...


In [ ]:
database = {"user":{},
    "flightPlans":pd.DataFrame(get_system_locations("OE")),
    "markets":pd.DataFrame(get_market_data('OE-PM')),
    "myShips":pandas.DataFrame({}),
    "myStructures":pandas.DataFrame({}),
    "locations":pandas.DataFrame({}),
    "goods":pandas.DataFrame({}),
    "loans":pandas.DataFrame({}),
    "structures":pandas.DataFrame({}),
    "ships":pandas.DataFrame({}),
    }

In [ ]:
def update_markets(location):
    if not ships_present(location): print(f"ERROR: NO SHIPS PRESENT AT {location}")
    else:
        data = get_market_data(location)
        timestamp = str(datetime.date.today())
        for datum in data:
            datum['location'] = location
            datum['date'] = timestamp
        

In [66]:
#add location and date
location = 'OE-PM-TR'
data = get_market_data(location)['marketplace']



In [2]:
get_market_data('OE-PM-TR')

[{'symbol': 'METALS',
  'volumePerUnit': 1,
  'pricePerUnit': 2,
  'spread': 1,
  'purchasePricePerUnit': 3,
  'sellPricePerUnit': 1,
  'quantityAvailable': 93751},
 {'symbol': 'CONSTRUCTION_MATERIALS',
  'volumePerUnit': 1,
  'pricePerUnit': 140,
  'spread': 4,
  'purchasePricePerUnit': 144,
  'sellPricePerUnit': 136,
  'quantityAvailable': 0},
 {'symbol': 'FUEL',
  'volumePerUnit': 1,
  'pricePerUnit': 2,
  'spread': 1,
  'purchasePricePerUnit': 3,
  'sellPricePerUnit': 1,
  'quantityAvailable': 87271},
 {'symbol': 'FUSION_REACTORS',
  'volumePerUnit': 6,
  'pricePerUnit': 20400,
  'spread': 196,
  'purchasePricePerUnit': 20596,
  'sellPricePerUnit': 20204,
  'quantityAvailable': 0}]

In [81]:
str(datetime.date.today())

'2022-07-17'

In [75]:
timestamp = str(datetime.date.today())
for datum in data:
    datum['location'] = location
    datum['date'] = timestamp

In [76]:
data

[{'symbol': 'METALS',
  'volumePerUnit': 1,
  'pricePerUnit': 2,
  'spread': 1,
  'purchasePricePerUnit': 3,
  'sellPricePerUnit': 1,
  'quantityAvailable': 93751,
  'location': 'OE-PM-TR',
  'date': datetime.datetime(2022, 7, 17, 12, 9, 42, 437131)},
 {'symbol': 'CONSTRUCTION_MATERIALS',
  'volumePerUnit': 1,
  'pricePerUnit': 140,
  'spread': 4,
  'purchasePricePerUnit': 144,
  'sellPricePerUnit': 136,
  'quantityAvailable': 0,
  'location': 'OE-PM-TR',
  'date': datetime.datetime(2022, 7, 17, 12, 9, 42, 437131)},
 {'symbol': 'FUEL',
  'volumePerUnit': 1,
  'pricePerUnit': 2,
  'spread': 1,
  'purchasePricePerUnit': 3,
  'sellPricePerUnit': 1,
  'quantityAvailable': 87271,
  'location': 'OE-PM-TR',
  'date': datetime.datetime(2022, 7, 17, 12, 9, 42, 437131)},
 {'symbol': 'FUSION_REACTORS',
  'volumePerUnit': 6,
  'pricePerUnit': 20400,
  'spread': 196,
  'purchasePricePerUnit': 20596,
  'sellPricePerUnit': 20204,
  'quantityAvailable': 0,
  'location': 'OE-PM-TR',
  'date': datetime.